In [1]:
import numpy as np
import cv2
from tqdm import tqdm #Progress bar
import os
TRAIN_IMAGE_DIR = '../input/train/images/' #img_id is x(input)
TRAIN_MASK_DIR = '../input/train/masks/'   #rle_mask is y(output)
TEST_IMAGE_DIR = '../input/test/images/'

train_d = os.listdir(TRAIN_IMAGE_DIR) 

In [2]:
x = [np.array(cv2.imread(TRAIN_IMAGE_DIR + p, cv2.IMREAD_GRAYSCALE), dtype=np.uint8) for p in tqdm(train_d)] #cv2.imread=openCV image read
x = np.array(x)/255

y = [np.array(cv2.imread(TRAIN_MASK_DIR + p, cv2.IMREAD_GRAYSCALE), dtype=np.uint8) for p in tqdm(train_d)]
y = np.array(y)/255
print(x.shape,y.shape)

100%|██████████| 4000/4000 [00:03<00:00, 1117.70it/s]


(4000, 101, 101) (4000, 101, 101)


In [3]:
x=np.expand_dims(x,axis=3) #EXPAND DIM OF X AND INSERT NEW AXIS @ 3 
y=np.expand_dims(y,axis=3)
print(x.shape,y.shape)

(4000, 101, 101, 1) (4000, 101, 101, 1)


In [4]:
from keras.layers import MaxPooling2D,Conv2D,Dense,Dropout,Input,Conv2DTranspose,Concatenate
from keras.models import Sequential,Model
def conv_block(num_layers,inp,units,kernel_size):
    x = input
    for l in range(num_layers): #repeat 32-24-16 ----4 times
        x = Conv2D(units, kernel_size=kernel_size,padding='SAME',activation='relu')(x)
    return x
input = Input(shape=(101,101,1))
cnn1 = conv_block(4,input,32,3)
cnn2 = conv_block(4,input,24,5)
cnn3 = conv_block(4,input,16,7)
concat = Concatenate()([cnn1,cnn2,cnn3])
d1 = Conv2D(16,1,activation='relu')(concat)
out = Conv2D(1,1,activation='sigmoid')(d1) #filter_size = 1 ,so that 1x1 filter will scan over for more learning

model = Model(inputs=[input], outputs=[out])
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
model.summary() # start_dim=(101,101,1) == #end_dim=(101,101,1)

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 101, 101, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 101, 101, 32) 320         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 101, 101, 24) 624         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 101, 101, 16) 800         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (

In [5]:
model.fit(x,y,epochs=50,batch_size=200,validation_split=0.2,verbose=1)

Train on 3200 samples, validate on 800 samples
Epoch 1/50
3200/3200 [==============================] - 26s 8ms/step - loss: 0.6015 - acc: 0.7226 - val_loss: 0.5650 - val_acc: 0.7532
Epoch 2/50
3200/3200 [==============================] - 19s 6ms/step - loss: 0.5619 - acc: 0.7517 - val_loss: 0.5434 - val_acc: 0.7532
Epoch 3/50
3200/3200 [==============================] - 19s 6ms/step - loss: 0.5270 - acc: 0.7517 - val_loss: 0.5109 - val_acc: 0.7532
Epoch 4/50
3200/3200 [==============================] - 19s 6ms/step - loss: 0.4983 - acc: 0.7517 - val_loss: 0.4868 - val_acc: 0.7532
Epoch 5/50
3200/3200 [==============================] - 19s 6ms/step - loss: 0.4838 - acc: 0.7517 - val_loss: 0.4599 - val_acc: 0.7532
Epoch 6/50
3200/3200 [==============================] - 19s 6ms/step - loss: 0.4662 - acc: 0.7694 - val_loss: 0.4722 - val_acc: 0.8145
Epoch 7/50
3200/3200 [==============================] - 19s 6ms/step - loss: 0.4583 - acc: 0.8005 - val_loss: 0.4204 - val_acc: 0.8219
Epoch 8/

In [6]:
#test_data
test_d=os.listdir(TEST_IMAGE_DIR)

x_test = [np.array(cv2.imread(TEST_IMAGE_DIR + p, cv2.IMREAD_GRAYSCALE), dtype=np.uint8) for p in tqdm(test_d)]
x_test = np.array(x_test)/255
print(x_test.shape)
x_test = np.expand_dims(x_test,axis=3)
print(x_test.shape)

100%|██████████| 18000/18000 [00:22<00:00, 811.47it/s]


(18000, 101, 101)
(18000, 101, 101, 1)


In [7]:
predict=model.predict(x_test,verbose=1)

18000/18000 [==============================] - 39s 2ms/step


In [8]:
#copy-pasted for rendering
def RLenc(img, order='F', format=True):
    """
    img is binary mask image, shape (r,c)
    order is down-then-right, i.e. Fortran
    format determines if the order needs to be preformatted (according to submission rules) or not

    returns run length as an array or string (if format is True)
    """
    bytes = img.reshape(img.shape[0] * img.shape[1], order=order)
    runs = []  ## list of run lengths
    r = 0  ## the current run length
    pos = 1  ## count starts from 1 per WK
    for c in bytes:
        if (c == 0):
            if r != 0:
                runs.append((pos, r))
                pos += r
                r = 0
            pos += 1
        else:
            r += 1
            
    # if last run is unsaved (i.e. data ends with 1)
    if r != 0:
        runs.append((pos, r))
        pos += r
        r = 0

    if format:
        z = ''

        for rr in runs:
            z += '{} {} '.format(rr[0], rr[1])
        return z[:-1]
    else:
        return runs

pred_dict = {fn[:-4]:RLenc(np.round(predict[i,:,:,0])) for i,fn in tqdm(enumerate(test_d))}

18000it [06:45, 44.41it/s]


In [14]:
import pandas as pd
sub = pd.DataFrame.from_dict(pred_dict,orient='index')
sub.index.names = ['id']
sub.columns = ['rle_mask']
sub.to_csv('submission.csv')
